# <div align = "center" style="color:rgb(10, 250, 150);"> Dima Data </div>

10aug_snapshot	ALL the users who did not have any disbursed loans until 10th Aug 2025

# Define Libraries

In [1]:

# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.
# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
from google.cloud import storage
import os

import time
from datetime import datetime
import uuid
import joblib
import uuid

import gcsfs
import duckdb as dd



path = r'C:\Users\Dwaipayan\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')
os.environ["GOOGLE_CLOUD_PROJECT"] = "prj-prod-dataplatform"
# %% [markdown]
## Configure Settings
# Set options or configurations as needed
pd.set_option('display.max_columns', None)
pd.set_option("Display.max_rows", 100)


# Constant

In [2]:
CURRENT_DATE = datetime.now().strftime("%Y%m%d")


# <div align="left" style="color:rgb(51, 250, 250);"> Functions </div>

## <div align="left" style="color:rgb(51, 250, 250);"> Save the data to google clound storage </div>

In [3]:
def save_df_to_gcs(df, bucket_name, destination_blob_name, file_format='csv'):
    """Saves a pandas DataFrame to Google Cloud Storage.

    Args:
        df: The pandas DataFrame to save.
        bucket_name: The name of the GCS bucket.
        destination_blob_name: The name of the blob to be created.
        file_format: The file format to save the DataFrame in ('csv' or 'parquet').
    """

    # Create a temporary file
    if file_format == 'csv':
        temp_file = 'temp.csv'
        df.to_csv(temp_file, index=False)
    elif file_format == 'parquet':
        temp_file = 'temp.parquet'
        df.to_parquet(temp_file, index=False)
    else:
        raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

    # Upload the file to GCS
    storage_client = storage.Client(project="prj-prod-dataplatform")

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(temp_file)

    # Remove the temporary file
    import os
    os.remove(temp_file)
    


## <div align="left" style="color:rgb(51, 250, 250);"> Read the Data from Google Cloud Storage </div>

In [4]:
def read_df_from_gcs(bucket_name, source_blob_name, file_format='csv'):
    """Reads a DataFrame from Google Cloud Storage.

    Args:
        bucket_name: The name of the GCS bucket.
        source_blob_name: The name of the blob to read.
        file_format: The file format to read ('csv' or 'parquet').

    Returns:
        pandas.DataFrame: The data loaded from the GCS file.
    """
    # Create a temporary file name
    temp_file = f'temp.{file_format}'
    
    try:
        # Initialize GCS client
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(source_blob_name)

        # Download the file to a temporary location
        blob.download_to_filename(temp_file)

        # Read the file into a DataFrame
        if file_format == 'csv':
            df = pd.read_csv(temp_file, low_memory=False)
        elif file_format == 'parquet':
            df = pd.read_parquet(temp_file)
        else:
            raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

        return df

    finally:
        # Clean up the temporary file
        if os.path.exists(temp_file):
            os.remove(temp_file)

## <div align = "left" style="color:rgb(51, 250, 250);"> Data Quality Report </div>

In [5]:
def data_quality_report(df, target_col='ln_fspd30_flag'):
    # Initialize an empty list to store each row of data
    report_data = []
    # Iterate over each column in the DataFrame to compute metrics
    for col in df.columns:
        # Determine the data type of the column
        data_type = df[col].dtype
       
        # Calculate the number of missing values in the column
        missing_values = df[col].isnull().sum()
       
        # Calculate the percentage of missing values relative to the total number of rows
        missing_percentage = (missing_values / len(df)) * 100
       
        # Calculate the number of unique values in the column
        unique_values = df[col].nunique()
       
        # Calculate the percentage of non-missing values
        non_missing_percentage = ((len(df) - missing_values) / len(df)) * 100
       
        # Check if the column is numeric to compute additional metrics
        if pd.api.types.is_numeric_dtype(df[col]):
            # Compute minimum, maximum, mean, median, mode, mode percentage, standard deviation, and quantiles
            min_value = df[col].min()
            max_value = df[col].max()
            mean_value = df[col].mean()
            median_value = df[col].median()
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = df[col].std()
            quantile_25 = df[col].quantile(0.25)
            quantile_50 = df[col].quantile(0.50)  # Same as median
            quantile_75 = df[col].quantile(0.75)
            
            # Calculate the Interquartile Range (IQR)
            iqr = quantile_75 - quantile_25
            
            # Calculate Skewness and Kurtosis
            skewness = df[col].skew()
            kurtosis = df[col].kurt()
            
            # Calculate Coefficient of Variation (CV) - standardized measure of dispersion
            cv = (std_dev / mean_value) * 100 if mean_value != 0 else None
            
            # Calculate correlation with target variable if target exists in dataframe
            if target_col in df.columns and col != target_col and pd.api.types.is_numeric_dtype(df[target_col]):
                # Calculate correlation only using rows where both columns have non-null values
                correlation = df[[col, target_col]].dropna().corr().iloc[0, 1]
            else:
                correlation = None
        else:
            # Assign None for non-numeric columns where appropriate
            min_value = None
            max_value = None
            mean_value = None
            median_value = None
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else None
            mode_percentage = (df[col] == mode_value).sum() / len(df) * 100 if mode_value is not None else None
            std_dev = None
            quantile_25 = None
            quantile_50 = None
            quantile_75 = None
            iqr = None
            skewness = None
            kurtosis = None
            cv = None
            correlation = None
       
        # Append the computed metrics for the current column to the list
        report_data.append({
            'Column': col,
            'Data Type': data_type,
            'Missing Values': missing_values,
            'Missing Percentage': missing_percentage,
            'Unique Values': unique_values,
            'Min': min_value,
            'Max': max_value,
            'Mean': mean_value,
            'Median': median_value,
            'Mode': mode_value,
            'Mode Percentage': mode_percentage,
            'Std Dev': std_dev,
            'Non-missing Percentage': non_missing_percentage,
            '25% Quantile': quantile_25,
            '50% Quantile': quantile_50,
            '75% Quantile': quantile_75,
            'IQR': iqr,
            'Skewness': skewness,
            'Kurtosis': kurtosis,
            'CV (%)': cv,
            f'Correlation with {target_col}': correlation
        })
    # Create the DataFrame from the list of dictionaries
    report = pd.DataFrame(report_data)
   
    # Return the complete data quality report DataFrame
    return report

# <div align = "left" style="color:rgb(51,250,250);"> Upload pickle file to Google Cloud Storage Bucke </div>

In [6]:
def upload_to_gcs(bucket_name, source_file_path, destination_blob_name):
    """Uploads a file to Google Cloud Storage"""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    blob.upload_from_filename(source_file_path)
    print(f"File {source_file_path} uploaded to {bucket_name}/{destination_blob_name}")

In [7]:
import pickle
import io
from google.cloud import storage
def save_pickle_to_gcs(data, bucket_name, destination_blob_name):
    """
    Save any Python object as a pickle file to Google Cloud Storage
    
    Args:
        data: The Python object to pickle (DataFrame, dict, list, etc.)
        bucket_name: Name of the GCS bucket
        destination_blob_name: Path/filename in the bucket
    """
    # Initialize the GCS client
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    
    # Serialize the data to pickle format in memory
    pickle_buffer = io.BytesIO()
    pickle.dump(data, pickle_buffer)
    pickle_buffer.seek(0)
    
    # Upload the pickle data to GCS
    blob.upload_from_file(pickle_buffer, content_type='application/octet-stream')
    print(f"Pickle file uploaded to gs://{bucket_name}/{destination_blob_name}")

## worktable_data_analysis.Latest_credo_before_90DOB

In [12]:
sq = f""" 
create or replace table worktable_data_analysis.Latest_credo_before_90DOB as
with a1 as 
(select distinct customerId customerid, credo_ref_no, credo_inquiry_date , credo_inquiry_date_manila, onboarding_date, ostype, sourcetab sources
from
 (select distinct trench2.customer_id customerid,'NULL' digitalLoanAccountId,refno credo_ref_no,createdOn credo_inquiry_date, datetime(timestamp(createdOn), 'Asia/Manila') as credo_inquiry_date_manila ,date(ln_90DOB_date) as ln_90DOB_date, sourcetab, onboarding_date 
 ,OFDATECLOSED, coalesce(credolab_ostype.ostype,'android') ostype
 from
(select userId,refno,deviceId,createdOn,B.cust_id from  `dl_loans_db_raw.tdbk_credolab_track` A  join  `dl_customers_db_raw.tdbk_customer_mtb` B on A.userId=B.user_id )  tsa
join prj-prod-dataplatform.core_raw.customer_accounts ca on ca.OFCUSTOMERID = tsa.cust_id and ca.CRINTERCODE = 'POC20000'  and (ca.OFDATECLOSED = '1970-01-01' or ca.OFDATECLOSED is null)
join  ((select * from worktable_data_analysis.All_90DOB_snapshot)) trench2
on tsa.cust_id=trench2.customer_id
left join ((select distinct deviceId,'android' as ostype from credolab_raw.android_credolab_datasets_struct_columns
union all
select distinct deviceId,'ios' as ostype from  credolab_raw.ios_credolab_datasets_struct_columns)) credolab_ostype
 on credolab_ostype.deviceId=refno
 where  date(createdOn)<date(ln_90DOB_date)
 qualify row_number()over (partition by cust_id order by date(createdOn) desc )=1)A
)
select * from a1 where credo_ref_no is not null
 ;
 """

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table  worktable_data_analysis.Latest_credo_before_90DOB  created successfully')


Table  worktable_data_analysis.Latest_credo_before_90DOB  created successfully


## worktable_data_analysis.Latest_credo_before_180DOB

In [13]:
sq = f""" 
create or replace table worktable_data_analysis.Latest_credo_before_180DOB as
with a1 as 
(select 
distinct customerId customerid, credo_ref_no, credo_inquiry_date , credo_inquiry_date_manila, onboarding_date, ostype, sourcetab sources
from
 (select distinct trench2.customer_id customerid,'NULL' digitalLoanAccountId,refno credo_ref_no,createdOn credo_inquiry_date, datetime(timestamp(createdOn), 'Asia/Manila') as credo_inquiry_date_manila ,date(ln_180DOB_date) as ln_180DOB_date, sourcetab, onboarding_date 
 ,OFDATECLOSED,  coalesce(credolab_ostype.ostype,'android') ostype
 from
(select userId,refno,deviceId,createdOn,B.cust_id from  `dl_loans_db_raw.tdbk_credolab_track` A  join  `dl_customers_db_raw.tdbk_customer_mtb` B on A.userId=B.user_id )  tsa
join prj-prod-dataplatform.core_raw.customer_accounts ca on ca.OFCUSTOMERID = tsa.cust_id and ca.CRINTERCODE = 'POC20000'
 and (ca.OFDATECLOSED = '1970-01-01' or ca.OFDATECLOSED is null)
join  ((select * from worktable_data_analysis.All_180DOB_snapshot)) trench2
on tsa.cust_id=trench2.customer_id
left join ((select distinct deviceId,'android' as ostype from credolab_raw.android_credolab_datasets_struct_columns
union all
select distinct deviceId,'ios' as ostype from  credolab_raw.ios_credolab_datasets_struct_columns)) credolab_ostype
 on credolab_ostype.deviceId=refno
 where  date(createdOn)<date(ln_180DOB_date)
  qualify row_number()over (partition by cust_id order by date(createdOn) desc )=1)A
)
select * from a1 where credo_ref_no is not null
 ;
 """

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table  worktable_data_analysis.Latest_credo_before_180DOB  created successfully')


Table  worktable_data_analysis.Latest_credo_before_180DOB  created successfully


In [14]:
sq = f""" 
create or replace table worktable_data_analysis.Latest_credo_before_10augustsnapshot as
with a1 as 
(select customerId customerid, credo_ref_no, credo_inquiry_date , credo_inquiry_date_manila, onboarding_date, ostype, sourcetab sources
from
 (select distinct trench2.customer_id customerid,'NULL' digitalLoanAccountId,
 refno credo_ref_no,
 createdOn credo_inquiry_date, 
 datetime(timestamp(createdOn), 'Asia/Manila') as credo_inquiry_date_manila ,
 date(snapshot_date) as snapshot_date, 
 sourcetab, onboarding_date 
 ,OFDATECLOSED
 , coalesce(credolab_ostype.ostype,'android') ostype
 from
(select userId,refno,deviceId,createdOn,B.cust_id from  `dl_loans_db_raw.tdbk_credolab_track` A  join  `dl_customers_db_raw.tdbk_customer_mtb` B on A.userId=B.user_id )  tsa
join prj-prod-dataplatform.core_raw.customer_accounts ca on ca.OFCUSTOMERID = tsa.cust_id and ca.CRINTERCODE = 'POC20000'
and (ca.OFDATECLOSED = '1970-01-01' or ca.OFDATECLOSED is null)
join  ((select * from worktable_data_analysis.All_10thAugust_snapshot)) trench2
on tsa.cust_id=trench2.customer_id
left join ((select distinct deviceId,'android' as ostype from credolab_raw.android_credolab_datasets_struct_columns
union all
select distinct deviceId,'ios' as ostype from  credolab_raw.ios_credolab_datasets_struct_columns)) credolab_ostype
 on credolab_ostype.deviceId=refno
 where  date(createdOn)<date(snapshot_date)
 qualify row_number()over (partition by cust_id order by date(createdOn) desc )=1)A
)
select * from a1 where credo_ref_no is not null
 """

job = client.query(sq)
job.result()  # Wait for the job to complete.
time.sleep(5) # Delays for 30 seconds
print(f'Table  worktable_data_analysis.Latest_credo_before_10augustsnapshot  created successfully')


Table  worktable_data_analysis.Latest_credo_before_10augustsnapshot  created successfully


In [ ]:
# sq = """with a1 as 
# (select trench2.customer_id customerid, trench2.ln_90DOB_date, trench2.onboarding_date,   
# ca.OFCUSTOMERID, ca.CRINTERCODE, ca.OFDATECLOSED, case when ca.CRINTERCODE = 'POC20000' then 'TSA' else 'Other' end accounts
# from worktable_data_analysis.All_90DOB_snapshot trench2
# left join prj-prod-dataplatform.core_raw.customer_accounts ca on ca.OFCUSTOMERID = trench2.customer_id
# )
# select * from a1
# ;"""

# df = client.query(sq).to_dataframe()



In [ ]:
df['accounts'].value_counts(dropna=False)